In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
hyperdrive_exp = Experiment(workspace=ws, name="hyperdrive-params-optimization")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = hyperdrive_exp.start_logging()

Workspace name: studio-east-us
Azure region: eastus
Subscription id: 80ba468e-6001-414e-891d-cc0b450c24b1
Resource group: project-east-us


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "hyperdrive-cluster"

provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, provisioning_config)
compute_target.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os


# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.05, 0.1),
    '--max_iter': choice(25, 100, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=ps, 
                                     policy=policy, 
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [7]:
# Submit hyperdrive_run to the experiment
hyperdrive_run = hyperdrive_exp.submit(hyperdrive_config)

# Show run details with the widget
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
# Get your best run 
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

# View by parameters like in class
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization Strength:', best_run_metrics['Regularization Strength:'])
print('Max iterations:', best_run_metrics['Max iterations:'])
print('Parameter values', parameter_values)

Best Run Id:  HD_adb2f6c6-04b1-4324-a86f-37b791ed09ad_5
Accuracy: 0.9123925139099646
Regularization Strength: 0.07808353662770287
Max iterations: 200
Parameter values ['--C', '0.07808353662770287', '--max_iter', '200']


In [23]:
# Save the model from that run
best_run.download_file('outputs/model.joblib', 'outputs/hyperdrive-model.joblib')

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [15]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

training_data = pd.concat([x_train, y_train], axis=1)
validation_data= pd.concat([x_test, y_test], axis=1)

In [16]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    validation_data=validation_data,
    enable_early_stopping=True,
    max_concurrent_iterations=4,
    featurization='auto')

automl_exp = Experiment(ws, 'automl')

In [17]:
# Submit your automl run
automl_run = automl_exp.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_5a55b3e9-7393-46cf-bc20-949c5e85a132,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping


2023/12/05 15:46:18 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2590                          |1                               |23065                                 |
+------------------------------+--------------------------------+-----

2023-12-05:16:04:21,661 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [32]:
# Retrieve the best model
best_run_automl, fitted_model_automl = automl_run.get_output()
best_run_automl_metrics = best_run_automl.get_metrics()

In [64]:
# Verify Fitted Model
print(fitted_model_automl)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/moun...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=511, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=2.1875, reg_lambda=0.4166666666666667, subsample=0.5, tree_method='auto'))], verbose=False)), 

In [65]:
logistic_regression_metrics = fitted_model_automl.named_steps['prefittedsoftvotingclassifier'].estimators[5][1].named_steps['logisticregression']

print('Best Run Id: ', best_run_automl.id)
print('Accuracy:', best_run_automl_metrics['accuracy'])

# Regression stats to compare with HyperDrive
print('Regularization Strength: (ensemble) ', logistic_regression_metrics.C)
print('Max iterations: (ensemble)', logistic_regression_metrics.max_iter)

Best Run Id:  AutoML_5a55b3e9-7393-46cf-bc20-949c5e85a132_33
Accuracy: 0.9160343955488113
Regularization Strength: (ensemble)  51.79474679231202
Max iterations: (ensemble) 100


In [66]:
# Save best automl model
best_run_automl.download_file('outputs/model.pkl', 'outputs/automl-model.pkl')

In [67]:
# HyperDrive x AutoML - Accuracy
print('HyperDrive Accuracy:', best_run_metrics['Accuracy'])
print('AutoML Accuracy:', best_run_automl_metrics['accuracy'])

HyperDrive Accuracy: 0.9123925139099646
AutoML Accuracy: 0.9160343955488113
